In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [2]:
# Use BART model for summarization
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

c:\ana\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\ana\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:

# Load the dataset
df  = pd.read_csv(r"C:\Users\DELL\Desktop\samsum-test.csv")
dataset = Dataset.from_pandas(df)

In [4]:

# Preprocessing function
def preprocess(example):
    model_inputs = tokenizer(example["dialogue"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["summary"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_data = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

c:\ana\Lib\site-packages\transformers\tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [5]:
# Step 4: Train-test split
split = tokenized_data.train_test_split(test_size=0.2)
train_ds = split["train"]
eval_ds  = split["test"]

In [6]:
print(split.keys())


dict_keys(['train', 'test'])


In [7]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode token IDs back to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Very basic cleaning (strip spaces)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Simple matching: exact match (optional: can be improved later)
    exact_matches = [int(pred == label) for pred, label in zip(decoded_preds, decoded_labels)]
    acc = np.mean(exact_matches)

    # For F1 score, treat text strings as "labels"
    f1 = f1_score(decoded_labels, decoded_preds, average='macro', zero_division=1)

    return {
        "accuracy": acc,
        "f1": f1
    }


In [8]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    predict_with_generate=True,
    no_cuda=True,  
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,  # Use the "train" split
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
   
)

# Start training
trainer.train()


c:\ana\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/656 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5235, 'learning_rate': 4.75609756097561e-06, 'epoch': 1.52}
{'train_runtime': 10113.4852, 'train_samples_per_second': 0.13, 'train_steps_per_second': 0.065, 'train_loss': 0.44789900430818885, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=0.44789900430818885, metrics={'train_runtime': 10113.4852, 'train_samples_per_second': 0.13, 'train_steps_per_second': 0.065, 'train_loss': 0.44789900430818885, 'epoch': 2.0})